In [150]:
import spotipy
import pandas as pd
import numpy as np
from spotipy.oauth2 import SpotifyOAuth
import time

In [151]:
client_id='c4ff820f994745288df451ea7d6940d5'
client_secret='be2297719e5e4fb698f112c4374ee3b1'
redirect_uri='https://open.spotify.com/'
token_url = "https://accounts.spotify.com/api/token"

## User Auth
- access spotify api via OAuth workflow

In [152]:
scope = 'playlist-read-private'

sp_oauth = SpotifyOAuth(client_id=client_id,client_secret=client_secret,redirect_uri=redirect_uri,scope=scope)
token_info = sp_oauth.get_cached_token() 
if not token_info:
    auth_url = sp_oauth.get_authorize_url()
    print(auth_url)
    response = input('Paste the above link into your browser, then paste the redirect url here: ')

    code = sp_oauth.parse_response_code(response)
    token_info = sp_oauth.get_access_token(code)

    token = token_info['access_token']

sp = spotipy.Spotify(auth=token, retries=10, requests_timeout=10)

https://accounts.spotify.com/authorize?client_id=c4ff820f994745288df451ea7d6940d5&response_type=code&redirect_uri=https%3A%2F%2Fopen.spotify.com%2F&scope=playlist-read-private


/var/folders/dy/m7tb4mx52sv0lxfvtf95srbc0000gn/T/ipykernel_91399/4118829141.py:11: DeprecationWarning: You're using 'as_dict = True'.get_access_token will return the token string directly in future versions. Please adjust your code accordingly, or use get_cached_token instead.
  token_info = sp_oauth.get_access_token(code)


In [153]:
# scope = 'playlist-read-private'

# oauth = spotipy.util.prompt_for_user_token(client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri, scope=scope)
# sp = spotipy.Spotify(auth=oauth)

In [154]:
if sp:
    print(sp.me())

{'display_name': 'amaya lim', 'external_urls': {'spotify': 'https://open.spotify.com/user/nightrainlily'}, 'href': 'https://api.spotify.com/v1/users/nightrainlily', 'id': 'nightrainlily', 'images': [{'url': 'https://i.scdn.co/image/ab67757000003b82180bd1d0c567a1e26710aeb8', 'height': 64, 'width': 64}, {'url': 'https://i.scdn.co/image/ab6775700000ee85180bd1d0c567a1e26710aeb8', 'height': 300, 'width': 300}], 'type': 'user', 'uri': 'spotify:user:nightrainlily', 'followers': {'href': None, 'total': 365}}


## Getting Playlists
- get all playlists via current_user_playlist method. use offsets to avoid rate limiting
- filter playlists using regex for only project playlists (numbered)
- create pandas df for analysis

In [155]:
from time import sleep
from tqdm import tqdm, trange

In [156]:
def get_all_playlists():
    playlist_count = 1000
    results = []
    offset = 0
    pbar = tqdm(total=1000)
    while offset < playlist_count:
        results.append(sp.current_user_playlists(limit=50, offset=offset))
        time.sleep(10)
        pbar.update(50)
        offset += 50
    pbar.close()
    return results
results = get_all_playlists()

100%|██████████| 1000/1000 [03:29<00:00,  4.77it/s]


In [157]:
def parse(playlist_json, target):
    lst = []
    for i in np.arange(len(playlist_json['items'])):
        target_value = playlist_json['items'][i]['track'][target]
        lst.append(target_value)
    return lst

def artist_parse(playlist_json, target):
    lst = []
    p = parse(playlist_json, target)
    for i in np.arange(len(p)):
        target_value = p[i][0]['id']
        lst.append(target_value)
    return lst
    

In [182]:
import re

playlist_data = []

for result in results:
    # pbar = tqdm(total=1000)
    for playlist in result['items']:
        reg = "^([0-9])+\s([a-z]+(\s?)([a-z]?))"
        # pbar.update(1)
        if re.search(reg, playlist['name']) != None and playlist['public'] == True:
            track_info = sp.playlist_tracks(playlist['id'], fields='items(track(id, name, artists))')
            playlist_data.append({
                'name' : playlist['name'],
                'track_ids' : parse(track_info, 'id'),
                'track_names' : parse(track_info, 'name'),
                'track_artists' : artist_parse(track_info, 'artists'),
                'id' : playlist['id'],
                'url' : playlist['external_urls']['spotify'],
                'image' : playlist['images'][0]['url']
            })
    # pbar.close()

playlists = pd.DataFrame(playlist_data)
# playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})
# playlists.head()

In [183]:
def pretty_tracks(col, name):
        all_l = []
        for i in np.arange(len(playlists[col])):
                row = playlists[col].values[i]
                l = []
                for j in row:
                        for k in np.arange(len(row)):
                                track = item[k]['track']
                                id = track[name]
                                l.append(id)
                all_l.append(l)
        return all_l

# playlists['track_ids'] = pretty_tracks('track_ids', 'id')
# playlists['track_names'] = pretty_tracks('track_names', 'name')

# playlists.head().style.format({'image': lambda x : f'<img src="{x}" width="100px"/>'})
playlists.head()


,name,track_ids,track_names,track_artists,id,url,image
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...","[053q0ukIDRgzwTr4vNSwab, 6beUvFUlKliUYJdLOXNj9...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...
1,00 northern stars,"[4YjjNHtEsTX6Af4mCTupT5, 77Y57qRJBvkGCUw9qs0qM...","[Bless The Broken Road, In the Blood, Bruises ...","[0a1gHP0HAqALbEyxaD5Ngn, 0hEurMDQu99nJRq8pTxO1...",7h4ROQ5TPc1xQvZLqRm8Zb,https://open.spotify.com/playlist/7h4ROQ5TPc1x...,https://mosaic.scdn.co/640/ab67616d00001e02098...
2,844 what did you expect,"[0rxQTgJzd0HCUk4QUxSkjK, 1EvFahLuLZ0vSVZo8YR3G...","[Even If It Hurts (feat. Blood Orange), Due We...","[1xcMOgFUM1IYZE22YjCvsL, 0fEfMW5bypHZ0A8eLnhwj...",0KgwFInw8xWYmtDQWayf2p,https://open.spotify.com/playlist/0KgwFInw8xWY...,https://mosaic.scdn.co/640/ab67616d00001e02145...
3,842 begin again,"[5zjQ3gF7xeNhMh0Is6wX0b, 553klLGtJCONJztTXktVF...","[Edge Of The Ocean, Reality Mind, Hey Big Eyes...","[2TpNrO0G2ahji2IOR94TLA, 2mirb9SKAm6IUHtPwreoq...",0YhNq6NX9HpmGan7uIxQwc,https://open.spotify.com/playlist/0YhNq6NX9Hpm...,https://image-cdn-ak.spotifycdn.com/image/ab67...
4,841 stay true,"[45zrZUWz5XrAKk50u2N6Kl, 12UQIJePnGeLmpVReYpG2...","[Collapse, Sunny day, Getting Older, Gemini an...","[6eb2gxfoBSQhmDYkQKz93b, 35l9BRT7MXmM8bv2WDQiy...",6OrcGRAMqQ3A6o2zCmqTj5,https://open.spotify.com/playlist/6OrcGRAMqQ3A...,https://image-cdn-ak.spotifycdn.com/image/ab67...


In [184]:
artists = playlists.pop('track_artists')
artists

0      [053q0ukIDRgzwTr4vNSwab, 6beUvFUlKliUYJdLOXNj9...
1      [0a1gHP0HAqALbEyxaD5Ngn, 0hEurMDQu99nJRq8pTxO1...
2      [1xcMOgFUM1IYZE22YjCvsL, 0fEfMW5bypHZ0A8eLnhwj...
3      [2TpNrO0G2ahji2IOR94TLA, 2mirb9SKAm6IUHtPwreoq...
4      [6eb2gxfoBSQhmDYkQKz93b, 35l9BRT7MXmM8bv2WDQiy...
                             ...                        
918    [6xdRb2GypJ7DqnWAI2mHGn, 1r1uxoy19fzMxunt3ONAk...
919    [3WrFJ7ztbogyGnTHbHJFl2, 5B7NeaqVrmXPyF05C9tnZ...
920    [5QdyldG4Fl4TPiOIeMNpBZ, 3h9TfIgwhovQELlP2jj4x...
921    [2gqMBdyddvN82dzZt4ZF14, 2rDxtYUzTAYJJE3Bl3Z5I...
922    [4vAQ4M7vgItwBtmBTgRu48, 6Gl4Q3ePw6HKMfIOix5Qp...
Name: track_artists, Length: 923, dtype: object

In [185]:
def get_url(p_name):
    info = sp.playlist(playlists[playlists['name'] == p_name]['id'].item())
    url = info['external_urls']['spotify']
    return url

## Getting Song Data
- manipulate playlist df to get songs
- drop na values (locally stored files)

In [186]:
songs = playlists
songs['zipp'] = list(zip(songs['track_ids'], songs['track_names']))

In [187]:
def merge(z):
    return list(tuple(zip(z[0], z[1])))

songs['zipp'] = songs['zipp'].apply(merge)
songs.head()

,name,track_ids,track_names,id,url,image,zipp
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"[(71J2rawCGulCAyTMtNY3Ej, Delete Forever), (4S..."
1,00 northern stars,"[4YjjNHtEsTX6Af4mCTupT5, 77Y57qRJBvkGCUw9qs0qM...","[Bless The Broken Road, In the Blood, Bruises ...",7h4ROQ5TPc1xQvZLqRm8Zb,https://open.spotify.com/playlist/7h4ROQ5TPc1x...,https://mosaic.scdn.co/640/ab67616d00001e02098...,"[(4YjjNHtEsTX6Af4mCTupT5, Bless The Broken Roa..."
2,844 what did you expect,"[0rxQTgJzd0HCUk4QUxSkjK, 1EvFahLuLZ0vSVZo8YR3G...","[Even If It Hurts (feat. Blood Orange), Due We...",0KgwFInw8xWYmtDQWayf2p,https://open.spotify.com/playlist/0KgwFInw8xWY...,https://mosaic.scdn.co/640/ab67616d00001e02145...,"[(0rxQTgJzd0HCUk4QUxSkjK, Even If It Hurts (fe..."
3,842 begin again,"[5zjQ3gF7xeNhMh0Is6wX0b, 553klLGtJCONJztTXktVF...","[Edge Of The Ocean, Reality Mind, Hey Big Eyes...",0YhNq6NX9HpmGan7uIxQwc,https://open.spotify.com/playlist/0YhNq6NX9Hpm...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"[(5zjQ3gF7xeNhMh0Is6wX0b, Edge Of The Ocean), ..."
4,841 stay true,"[45zrZUWz5XrAKk50u2N6Kl, 12UQIJePnGeLmpVReYpG2...","[Collapse, Sunny day, Getting Older, Gemini an...",6OrcGRAMqQ3A6o2zCmqTj5,https://open.spotify.com/playlist/6OrcGRAMqQ3A...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"[(45zrZUWz5XrAKk50u2N6Kl, Collapse), (12UQIJeP..."


In [188]:
songs = songs.explode('zipp')
songs.head()

,name,track_ids,track_names,id,url,image,zipp
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(71J2rawCGulCAyTMtNY3Ej, Delete Forever)"
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(4Sn7ySAR30siowyB0jWqqX, I Hate Myself, I Want..."
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(0tP8FKhJsar5y4JcOH4Rjp, You Lose!)"
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(5YIVVzQcJG7pFhyNo0Ytlh, Door)"
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(5n06Jroq3ruFU5WJYnN8JO, Country)"


In [189]:
songs['track_name'] = songs['zipp'].map(lambda x: x[1])
songs['track_id'] = songs['zipp'].map(lambda x: x[0])
songs.head()

,name,track_ids,track_names,id,url,image,zipp,track_name,track_id
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(71J2rawCGulCAyTMtNY3Ej, Delete Forever)",Delete Forever,71J2rawCGulCAyTMtNY3Ej
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(4Sn7ySAR30siowyB0jWqqX, I Hate Myself, I Want...","I Hate Myself, I Want To Party",4Sn7ySAR30siowyB0jWqqX
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(0tP8FKhJsar5y4JcOH4Rjp, You Lose!)",You Lose!,0tP8FKhJsar5y4JcOH4Rjp
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(5YIVVzQcJG7pFhyNo0Ytlh, Door)",Door,5YIVVzQcJG7pFhyNo0Ytlh
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(5n06Jroq3ruFU5WJYnN8JO, Country)",Country,5n06Jroq3ruFU5WJYnN8JO


In [190]:
songs.reset_index()

,index,name,track_ids,track_names,id,url,image,zipp,track_name,track_id
0,0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(71J2rawCGulCAyTMtNY3Ej, Delete Forever)",Delete Forever,71J2rawCGulCAyTMtNY3Ej
1,0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(4Sn7ySAR30siowyB0jWqqX, I Hate Myself, I Want...","I Hate Myself, I Want To Party",4Sn7ySAR30siowyB0jWqqX
2,0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(0tP8FKhJsar5y4JcOH4Rjp, You Lose!)",You Lose!,0tP8FKhJsar5y4JcOH4Rjp
3,0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(5YIVVzQcJG7pFhyNo0Ytlh, Door)",Door,5YIVVzQcJG7pFhyNo0Ytlh
4,0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(5n06Jroq3ruFU5WJYnN8JO, Country)",Country,5n06Jroq3ruFU5WJYnN8JO
...,...,...,...,...,...,...,...,...,...,...
9388,922,"2022 releases i missed, whoopies!","[4euglpuBWHIqrTofPsdVTi, 337bOFFRgT6HyF2V4fLCY...","[Rotten, Mulholland Dr., Concrete Over Water, ...",1w1Jkxyxbd83lCaNeD1wkz,https://open.spotify.com/playlist/1w1Jkxyxbd83...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(1IHWl5LamUGEuP4ozKQSXZ, Tití Me Preguntó)",Tití Me Preguntó,1IHWl5LamUGEuP4ozKQSXZ
9389,922,"2022 releases i missed, whoopies!","[4euglpuBWHIqrTofPsdVTi, 337bOFFRgT6HyF2V4fLCY...","[Rotten, Mulholland Dr., Concrete Over Water, ...",1w1Jkxyxbd83lCaNeD1wkz,https://open.spotify.com/playlist/1w1Jkxyxbd83...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(3k2kIiJqUiIP49iUcOLpWT, The Place Where He In...",The Place Where He Inserted the Blade,3k2kIiJqUiIP49iUcOLpWT
9390,922,"2022 releases i missed, whoopies!","[4euglpuBWHIqrTofPsdVTi, 337bOFFRgT6HyF2V4fLCY...","[Rotten, Mulholland Dr., Concrete Over Water, ...",1w1Jkxyxbd83lCaNeD1wkz,https://open.spotify.com/playlist/1w1Jkxyxbd83...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(2kv9BIaOOHZrIphec1U68O, Soft Engine)",Soft Engine,2kv9BIaOOHZrIphec1U68O
9391,922,"2022 releases i missed, whoopies!","[4euglpuBWHIqrTofPsdVTi, 337bOFFRgT6HyF2V4fLCY...","[Rotten, Mulholland Dr., Concrete Over Water, ...",1w1Jkxyxbd83lCaNeD1wkz,https://open.spotify.com/playlist/1w1Jkxyxbd83...,https://image-cdn-ak.spotifycdn.com/image/ab67...,"(0I5OHUf0S6WoahmWRYHwEw, The Tisburys (On Main...",The Tisburys (On Main Street),0I5OHUf0S6WoahmWRYHwEw


In [181]:
songs = songs.dropna()

### Audio Features
- manipulate song df to get audio features from spotify api
- merge with song df for searchability
- take mean audio features as score for playlist overall
- merge with playlist df
- export dfs as csvs to avoid repeatedly calling api for analysis

In [117]:
def batch(lst, n):
    for i in range(0, len(lst), n):
        if len(lst) >= (i+n):
            yield lst[i:i + n]
        else:
            yield lst[i:]

def get_audio_features(uris):
    results = []
    pbar = tqdm(total=1000)
    batched = batch(uris, 100)
    for each in batched:
        results.append(sp.audio_features(each))
        pbar.update(50)
    pbar.close()
    return results

In [21]:
audio_features_results = get_audio_features(songs['track_id'])

4900it [00:23, 209.40it/s]                          


In [22]:
unpacked = [song for hundred in audio_features_results for song in hundred]

In [23]:
features = pd.DataFrame(unpacked)
features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.560,0.736,8,-7.640,1,0.0339,0.013500,0.099100,0.174,0.389,94.987,audio_features,71J2rawCGulCAyTMtNY3Ej,spotify:track:71J2rawCGulCAyTMtNY3Ej,https://api.spotify.com/v1/tracks/71J2rawCGulC...,https://api.spotify.com/v1/audio-analysis/71J2...,237481,4
1,0.361,0.574,9,-8.134,1,0.1080,0.331000,0.000213,0.117,0.748,79.334,audio_features,4Sn7ySAR30siowyB0jWqqX,spotify:track:4Sn7ySAR30siowyB0jWqqX,https://api.spotify.com/v1/tracks/4Sn7ySAR30si...,https://api.spotify.com/v1/audio-analysis/4Sn7...,234990,4
2,0.628,0.909,9,-6.168,0,0.0423,0.000068,0.372000,0.923,0.164,120.010,audio_features,0tP8FKhJsar5y4JcOH4Rjp,spotify:track:0tP8FKhJsar5y4JcOH4Rjp,https://api.spotify.com/v1/tracks/0tP8FKhJsar5...,https://api.spotify.com/v1/audio-analysis/0tP8...,204391,4
3,0.684,0.438,3,-8.309,1,0.0388,0.222000,0.000000,0.262,0.253,123.997,audio_features,5YIVVzQcJG7pFhyNo0Ytlh,spotify:track:5YIVVzQcJG7pFhyNo0Ytlh,https://api.spotify.com/v1/tracks/5YIVVzQcJG7p...,https://api.spotify.com/v1/audio-analysis/5YIV...,322563,4
4,0.581,0.650,1,-7.531,1,0.0280,0.005290,0.059000,0.125,0.199,104.989,audio_features,33qgZ7mWvlH95efOGQqfzZ,spotify:track:33qgZ7mWvlH95efOGQqfzZ,https://api.spotify.com/v1/tracks/33qgZ7mWvlH9...,https://api.spotify.com/v1/audio-analysis/33qg...,244691,4


In [90]:
songs_features = songs.merge(features,  left_on='track_id', right_on='id')
songs_features.head()

,name,track_ids,track_names,id_x,url,image,zipp,track_name,track_id,danceability,...,liveness,valence,tempo,type,id_y,uri,track_href,analysis_url,duration_ms,time_signature
0,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(71J2rawCGulCAyTMtNY3Ej, Delete Forever)",Delete Forever,71J2rawCGulCAyTMtNY3Ej,0.560,...,0.174,0.389,94.987,audio_features,71J2rawCGulCAyTMtNY3Ej,spotify:track:71J2rawCGulCAyTMtNY3Ej,https://api.spotify.com/v1/tracks/71J2rawCGulC...,https://api.spotify.com/v1/audio-analysis/71J2...,237481,4
1,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(71J2rawCGulCAyTMtNY3Ej, Delete Forever)",Delete Forever,71J2rawCGulCAyTMtNY3Ej,0.560,...,0.174,0.389,94.987,audio_features,71J2rawCGulCAyTMtNY3Ej,spotify:track:71J2rawCGulCAyTMtNY3Ej,https://api.spotify.com/v1/tracks/71J2rawCGulC...,https://api.spotify.com/v1/audio-analysis/71J2...,237481,4
2,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(4Sn7ySAR30siowyB0jWqqX, I Hate Myself, I Want...","I Hate Myself, I Want To Party",4Sn7ySAR30siowyB0jWqqX,0.361,...,0.117,0.748,79.334,audio_features,4Sn7ySAR30siowyB0jWqqX,spotify:track:4Sn7ySAR30siowyB0jWqqX,https://api.spotify.com/v1/tracks/4Sn7ySAR30si...,https://api.spotify.com/v1/audio-analysis/4Sn7...,234990,4
3,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(4Sn7ySAR30siowyB0jWqqX, I Hate Myself, I Want...","I Hate Myself, I Want To Party",4Sn7ySAR30siowyB0jWqqX,0.361,...,0.117,0.748,79.334,audio_features,4Sn7ySAR30siowyB0jWqqX,spotify:track:4Sn7ySAR30siowyB0jWqqX,https://api.spotify.com/v1/tracks/4Sn7ySAR30si...,https://api.spotify.com/v1/audio-analysis/4Sn7...,234990,4
4,00 sonnet,"[71J2rawCGulCAyTMtNY3Ej, 4Sn7ySAR30siowyB0jWqq...","[Delete Forever, I Hate Myself, I Want To Part...",4w5eygTTimXG091FHLE7zw,https://open.spotify.com/playlist/4w5eygTTimXG...,https://mosaic.scdn.co/640/ab67616d00001e020e8...,"(0tP8FKhJsar5y4JcOH4Rjp, You Lose!)",You Lose!,0tP8FKhJsar5y4JcOH4Rjp,0.628,...,0.923,0.164,120.010,audio_features,0tP8FKhJsar5y4JcOH4Rjp,spotify:track:0tP8FKhJsar5y4JcOH4Rjp,https://api.spotify.com/v1/tracks/0tP8FKhJsar5...,https://api.spotify.com/v1/audio-analysis/0tP8...,204391,4


In [99]:
playlist_features = songs_features[['name', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo']].groupby('name', sort=False).median()
playlist_features.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
name,,,,,,,,,,,
00 sonnet,0.5600,0.574,8.0,-8.1340,1.0,0.03880,0.2220,0.000713,0.1900,0.2530,110.0480
00 northern stars,0.5255,0.476,6.0,-7.6300,1.0,0.03130,0.6200,0.000016,0.1165,0.4230,133.7480
842 what did you expect,0.6430,0.546,6.0,-8.5965,0.0,0.05425,0.3440,0.015845,0.1670,0.5265,119.5205
842 begin again,0.4905,0.517,5.5,-9.1100,1.0,0.03045,0.3100,0.115500,0.1195,0.3245,108.1455
841 stay true,0.6515,0.636,6.0,-7.3915,0.0,0.03300,0.1227,0.065600,0.1085,0.5590,94.4955


### Genre

In [191]:
def get_genre(playlist_artists):
    results = []
    if len(playlist_artists) <= 50:
        artists = sp.artists(playlist_artists)
        for artist in artists['artists']:
            genres = artist['genres']
            results.append(genres)
    else:
        playlist_artists_batched = batch(playlist_artists, len(playlist_artists)//50 + 1)
        for artist_batch in playlist_artists_batched:
            artists = sp.artists(artist_batch)
            for artist in artists['artists']:
                genres = artist['genres']
                results.append(genres)
    time.sleep(1)
    return [genre for genres in results for genre in genres]

In [192]:
filtered_artists = pd.Series([list(filter(lambda artist: artist is not None, p_artists)) for p_artists in artists])
filtered_artists

0      [053q0ukIDRgzwTr4vNSwab, 6beUvFUlKliUYJdLOXNj9...
1      [0a1gHP0HAqALbEyxaD5Ngn, 0hEurMDQu99nJRq8pTxO1...
2      [1xcMOgFUM1IYZE22YjCvsL, 0fEfMW5bypHZ0A8eLnhwj...
3      [2TpNrO0G2ahji2IOR94TLA, 2mirb9SKAm6IUHtPwreoq...
4      [6eb2gxfoBSQhmDYkQKz93b, 35l9BRT7MXmM8bv2WDQiy...
                             ...                        
918    [6xdRb2GypJ7DqnWAI2mHGn, 1r1uxoy19fzMxunt3ONAk...
919    [3WrFJ7ztbogyGnTHbHJFl2, 5B7NeaqVrmXPyF05C9tnZ...
920    [5QdyldG4Fl4TPiOIeMNpBZ, 3h9TfIgwhovQELlP2jj4x...
921    [2gqMBdyddvN82dzZt4ZF14, 2rDxtYUzTAYJJE3Bl3Z5I...
922    [4vAQ4M7vgItwBtmBTgRu48, 6Gl4Q3ePw6HKMfIOix5Qp...
Length: 923, dtype: object

In [193]:
genres = playlist_features.copy().reset_index()
genres.head()

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo
0,00 sonnet,0.5600,0.574,8.0,-8.1340,1.0,0.03880,0.2220,0.000713,0.1900,0.2530,110.0480
1,00 northern stars,0.5255,0.476,6.0,-7.6300,1.0,0.03130,0.6200,0.000016,0.1165,0.4230,133.7480
2,842 what did you expect,0.6430,0.546,6.0,-8.5965,0.0,0.05425,0.3440,0.015845,0.1670,0.5265,119.5205
3,842 begin again,0.4905,0.517,5.5,-9.1100,1.0,0.03045,0.3100,0.115500,0.1195,0.3245,108.1455
4,841 stay true,0.6515,0.636,6.0,-7.3915,0.0,0.03300,0.1227,0.065600,0.1085,0.5590,94.4955


In [194]:
genres = pd.concat([genres, filtered_artists.rename('track_artists')], axis=1)
genres

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_artists
0,00 sonnet,0.5600,0.574,8.0,-8.1340,1.0,0.03880,0.2220,0.000713,0.1900,0.2530,110.0480,"[053q0ukIDRgzwTr4vNSwab, 6beUvFUlKliUYJdLOXNj9..."
1,00 northern stars,0.5255,0.476,6.0,-7.6300,1.0,0.03130,0.6200,0.000016,0.1165,0.4230,133.7480,"[0a1gHP0HAqALbEyxaD5Ngn, 0hEurMDQu99nJRq8pTxO1..."
2,842 what did you expect,0.6430,0.546,6.0,-8.5965,0.0,0.05425,0.3440,0.015845,0.1670,0.5265,119.5205,"[1xcMOgFUM1IYZE22YjCvsL, 0fEfMW5bypHZ0A8eLnhwj..."
3,842 begin again,0.4905,0.517,5.5,-9.1100,1.0,0.03045,0.3100,0.115500,0.1195,0.3245,108.1455,"[2TpNrO0G2ahji2IOR94TLA, 2mirb9SKAm6IUHtPwreoq..."
4,841 stay true,0.6515,0.636,6.0,-7.3915,0.0,0.03300,0.1227,0.065600,0.1085,0.5590,94.4955,"[6eb2gxfoBSQhmDYkQKz93b, 35l9BRT7MXmM8bv2WDQiy..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,2023 wrapped,0.5525,0.469,3.5,-8.9655,1.0,0.03950,0.4520,0.000309,0.1140,0.3840,114.9550,NaN
924,800 no relation (unedited),0.5505,0.493,5.0,-9.0870,1.0,0.03610,0.5200,0.000726,0.1160,0.3590,114.0290,NaN
925,2022 wrapped,0.5560,0.442,4.0,-9.4510,1.0,0.03570,0.5240,0.001260,0.1110,0.3960,123.9640,NaN
926,"2022 releases i missed, whoopies!",0.5740,0.587,6.0,-7.6710,1.0,0.03720,0.2690,0.001070,0.1260,0.5000,123.3440,NaN


In [136]:
genres_batched = np.array_split(genres, len(genres)//4 + 1)

genres_list = []

for batch in genres_batched:
    genres_list.append(batch['track_artists'].apply(get_genre))

genres_list

/Users/amayalim/Projects/turntable/.venv/lib/python3.11/site-packages/numpy/core/fromnumeric.py:59: FutureWarning: 'DataFrame.swapaxes' is deprecated and will be removed in a future version. Please use 'DataFrame.transpose' instead.
  return bound(*args, **kwds)


KeyboardInterrupt: 

In [137]:
flattened = [playlist_genres for batch in genres_list for playlist_genres in batch]

In [138]:
genres['genres'] = flattened

ValueError: Length of values (808) does not match length of index (928)

In [ ]:
genres

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_artists,genres
0,00 all yours,0.5830,0.5690,7.0,-7.9775,1.0,0.03600,0.37000,0.000694,0.11100,0.5520,114.7475,"[053q0ukIDRgzwTr4vNSwab, 6beUvFUlKliUYJdLOXNj9...","[art pop, canadian electropop, grave wave, ind..."
1,00 alright,0.6120,0.7020,2.0,-5.7070,1.0,0.05280,0.09080,0.000022,0.08060,0.5020,122.8040,"[0a1gHP0HAqALbEyxaD5Ngn, 0hEurMDQu99nJRq8pTxO1...","[contemporary country, country, country road, ..."
2,00 another dream,0.7140,0.6300,5.0,-6.3450,1.0,0.03630,0.13500,0.000007,0.13200,0.7775,115.9975,"[1xcMOgFUM1IYZE22YjCvsL, 0fEfMW5bypHZ0A8eLnhwj...","[alternative r&b, art pop, escape room, gauze ..."
3,00 before you go,0.5180,0.3410,5.0,-9.0890,1.0,0.04115,0.62700,0.000000,0.11050,0.2330,124.1130,"[2TpNrO0G2ahji2IOR94TLA, 2mirb9SKAm6IUHtPwreoq...","[modern dream pop, art pop, escape room, indie..."
4,00 call it love,0.5510,0.7580,2.0,-7.3850,1.0,0.03520,0.08780,0.000099,0.09600,0.5470,118.0200,"[6eb2gxfoBSQhmDYkQKz93b, 35l9BRT7MXmM8bv2WDQiy...","[latinx alternative, portland indie, bedroom p..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
923,95 time's up,0.6095,0.7145,1.5,-6.2555,1.0,0.20800,0.06975,0.000000,0.12850,0.5075,107.0790,"[1hLiboQ98IQWhpKeP9vRFw, 3pGNoAJX3UVctU6X8eG55...",None
924,96 curse the sun,0.6320,0.6625,6.0,-8.6790,1.0,0.05835,0.23200,0.005005,0.09035,0.4845,115.0190,"[3pGNoAJX3UVctU6X8eG55h, 3IunaFjvNKj98JW89JYv9...",None
925,97 rejoice,0.6765,0.7340,8.0,-5.9230,0.5,0.12650,0.11955,0.000003,0.12100,0.3460,117.5340,"[0nnYdIpahs41QiZ9MWp5Wx, 5hAhrnb0Ch4ODwWu4tsbp...",None
926,98 the shape we make,0.6030,0.5090,4.0,-6.8240,1.0,0.03410,0.37100,0.000197,0.10900,0.5720,121.0320,"[4vAQ4M7vgItwBtmBTgRu48, 6Gl4Q3ePw6HKMfIOix5Qp...",None


In [55]:
genres['genres'] = genres['track_artists'].apply(get_genre)
# genres_col = genres['track_artists'].apply(get_genre)
genres

Max Retries reached


SpotifyException: http status: 429, code:-1 - /v1/artists/?ids=1hLiboQ98IQWhpKeP9vRFw,3pGNoAJX3UVctU6X8eG55h,3IunaFjvNKj98JW89JYv9u,2hUYKu1x0UZQXvzCmggvSn,3IunaFjvNKj98JW89JYv9u,3kzwYV3OCB010YfXMF0Avt,2mirb9SKAm6IUHtPwreoqN,3ir2pF2mkiEWqyPenKTh5e,25uiPmTg16RbhZWAqwLBy5,0knGpCTbmG4ctl1wzYRZs4,1Uk1GyijF6fSfX4mWq5bfR,4hmP7SKOIhC8e1PZo8UG7f,7hp6PmppZj6iiolLVT4iEZ,0knGpCTbmG4ctl1wzYRZs4,6lcwlkAjBPSKnFBZjjZFJs,2JFTRDi5v7JtqoouVe1z5D,7hp6PmppZj6iiolLVT4iEZ,2mirb9SKAm6IUHtPwreoqN,1hLiboQ98IQWhpKeP9vRFw,2JFTRDi5v7JtqoouVe1z5D,7hp6PmppZj6iiolLVT4iEZ,4V8LLVI7PbaPR0K2TGSxFF,2hUYKu1x0UZQXvzCmggvSn,74KM79TiuVKeVCqs8QtB0B,6O4EGCCb6DoIiR6B1QCQgp,4kIwETcbpuFgRukE8o7Opx,6lcwlkAjBPSKnFBZjjZFJs,0knGpCTbmG4ctl1wzYRZs4,6CWTBjOJK75cTE8Xv8u1kj,3WGpXCj9YhhfX11TToZcXP,09s6bLEw45wioK9ytf3nsA,5mZC7ndY6oGMxJentRwsuV,04vj3iPUiVh5melWr0w3xT,2QoU3awHVdcHS8LrZEKvSM,4Ui2kfOqGujY81UcPrb5KE,0nnYdIpahs41QiZ9MWp5Wx,39vA9YljbnOApXKniLWBZv,25uiPmTg16RbhZWAqwLBy5,6CWTBjOJK75cTE8Xv8u1kj,0knGpCTbmG4ctl1wzYRZs4,2QoU3awHVdcHS8LrZ

In [57]:
result = genres.loc[(genres.track_artists == '1hLiboQ98IQWhpKeP9vRFw,3pGNoAJX3UVctU6X8eG55h,3IunaFjvNKj98JW89JYv9u,2hUYKu1x0UZQXvzCmggvSn,3IunaFjvNKj98JW89JYv9u,3kzwYV3OCB010YfXMF0Avt,2mirb9SKAm6IUHtPwreoqN,3ir2pF2mkiEWqyPenKTh5e,25uiPmTg16RbhZWAqwLBy5,0knGpCTbmG4ctl1wzYRZs4')]
result

,name,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,track_artists


In [66]:
print(sp.artist('3pGNoAJX3UVctU6X8eG55h'))

{'external_urls': {'spotify': 'https://open.spotify.com/artist/3pGNoAJX3UVctU6X8eG55h'}, 'followers': {'href': None, 'total': 10388}, 'genres': ['slacker rock'], 'href': 'https://api.spotify.com/v1/artists/3pGNoAJX3UVctU6X8eG55h', 'id': '3pGNoAJX3UVctU6X8eG55h', 'images': [{'url': 'https://i.scdn.co/image/ab6761610000e5eb0c560e617e4e4a57fc34ecf4', 'height': 640, 'width': 640}, {'url': 'https://i.scdn.co/image/ab676161000051740c560e617e4e4a57fc34ecf4', 'height': 320, 'width': 320}, {'url': 'https://i.scdn.co/image/ab6761610000f1780c560e617e4e4a57fc34ecf4', 'height': 160, 'width': 160}], 'name': 'Greg Mendez', 'popularity': 35, 'type': 'artist', 'uri': 'spotify:artist:3pGNoAJX3UVctU6X8eG55h'}


In [28]:
playlists.to_csv('csv_files/playlists.csv', index=False)
songs.to_csv('csv_files/songs.csv', index=False)
features.to_csv('csv_files/features.csv', index=False)

In [29]:
playlist_features.to_csv('csv_files/playlist_features.csv', index=True)